# Deploying a spaCy (text classificastion) model on Verta

Within Verta, a "Model" can be any arbitrary function: a traditional ML model (e.g., sklearn, PyTorch, TF, etc); a function (e.g., squaring a number, making a DB function etc.); or a mixture of the above (e.g., pre-processing code, a DB call, and then a model application.) See more [here](https://docs.verta.ai/verta/registry/concepts).

This notebook provides an example of how to deploy a spaCy model on Verta as a Verta Standard Model by extending [VertaModelBase](https://verta.readthedocs.io/en/master/_autogen/verta.registry.VertaModelBase.html?highlight=VertaModelBase#verta.registry.VertaModelBase).

Updated for Verta version: 0.18.2

This walkthrough is based on [this spaCy tutorial](https://github.com/explosion/spaCy/blob/v2.3.7/examples/training/train_textcat.py).

Train a convolutional neural network text classifier on the
IMDB dataset, using the `TextCategorizer` component. The dataset will be loaded
automatically via Thinc's built-in dataset loader. The model is added to
`spacy.pipeline`, and predictions are available via `doc.cats`.

<a href="https://colab.research.google.com/github/VertaAI/examples/blob/main/deployment/spacy/text-classification-spacy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 0. Imports

In [1]:
!python -m spacy download en

In [2]:
from __future__ import print_function

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import pickle
import random

import numpy as np
import spacy
from spacy.util import minibatch, compounding
import thinc.extra.datasets

### 0.1 Verta import and setup

In [3]:
# restart your notebook if prompted on Colab
try:
    import verta
except ImportError:
    !pip install verta

In [4]:
# import os
# os.environ['VERTA_EMAIL'] = 
# os.environ['VERTA_DEV_KEY'] = 
# os.environ['VERTA_HOST']

In [5]:
import os
from verta import Client
from verta.utils import ModelAPI

client = Client(os.environ['VERTA_HOST'])

---

## 1. Log model

### 1.1 Prepare data

In [6]:
def load_data(limit=0, split=0.8):
    """Load data from the IMDB dataset."""
    # Partition off part of the dataset to train and test
    train_data, _ = thinc.extra.datasets.imdb()
    random.shuffle(train_data)    
    train_data = train_data[-limit:]
    texts, labels = zip(*train_data)
    cats = [{"POSITIVE": bool(y), "NEGATIVE": not bool(y)} for y in labels]
    split = int(len(train_data) * split)
    return (texts[:split], cats[:split]), (texts[split:], cats[split:])

In [7]:
num_samples = 1000
train_test_split = 0.8

# load the IMDB dataset
print("Loading IMDB data...")
(train_texts, train_cats), (dev_texts, dev_cats) = load_data(
    limit=num_samples,
    split=train_test_split,
)
print(
    "Using {} examples ({} training, {} evaluation)".format(
        num_samples, len(train_texts), len(dev_texts)
    )
)
train_data = list(zip(train_texts, [{"cats": cats} for cats in train_cats]))

In [8]:
# sample train data
train_data[:1]

### 1.2 Train model

In [9]:
def evaluate(tokenizer, textcat, texts, cats):
    """Evaluate with text data, calculates precision, recall and f score"""
    docs = (tokenizer(text) for text in texts)
    tp = 0.0  # True positives
    fp = 1e-8  # False positives
    fn = 1e-8  # False negatives
    tn = 0.0  # True negatives
    for i, doc in enumerate(textcat.pipe(docs)):
        gold = cats[i]
        for label, score in doc.cats.items():
            if label not in gold:
                continue
            if label == "NEGATIVE":
                continue
            if score >= 0.5 and gold[label] >= 0.5:
                tp += 1.0
            elif score >= 0.5 and gold[label] < 0.5:
                fp += 1.0
            elif score < 0.5 and gold[label] < 0.5:
                tn += 1
            elif score < 0.5 and gold[label] >= 0.5:
                fn += 1
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    if (precision + recall) == 0:
        f_score = 0.0
    else:
        f_score = 2 * (precision * recall) / (precision + recall)
    return {"textcat_p": precision, "textcat_r": recall, "textcat_f": f_score}

In [10]:
attrs = {
    "model": "en",
    "n_iter": 2, # epochs
    "n_texts": 500, # num of training samples
    "architecture": "simple_cnn",
    "num_samples": num_samples,
    "train_test_split": train_test_split,
    "dropout": 0.2
}

In [11]:
# using the basic en model
try:
    nlp = spacy.load(attrs['model'])  # load existing spaCy model
except OSError:
    nlp = spacy.blank(attrs['model'])  # create blank Language class
    print("Created blank '{}' model".format(attrs['model']))
else:
    print("Loaded model '{}'".format(nlp))

# add the text classifier to the pipeline if it doesn't exist
if "textcat" not in nlp.pipe_names:
    textcat = nlp.create_pipe(
        "textcat",
        config={
            "exclusive_classes": True,
            "architecture": attrs['architecture'],
        }
    )
    nlp.add_pipe(textcat, last=True)
# otherwise, get it, so we can add labels to it
else:
    textcat = nlp.get_pipe("textcat")

# add label to text classifier
_ = textcat.add_label("POSITIVE")
_ = textcat.add_label("NEGATIVE")

In [12]:
# get names of other pipes to disable them during training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "textcat"]
print("other pipes:", other_pipes)
with nlp.disable_pipes(*other_pipes):  # only train textcat
    optimizer = nlp.begin_training()
    print("Training the model...")
    print("{:^5}\t{:^5}\t{:^5}\t{:^5}".format("LOSS", "P", "R", "F"))
    batch_sizes = compounding(4.0, 32.0, 1.001)
    for i in range(attrs['n_iter']):
        losses = {}
        # batch up the examples using spaCy's minibatch
        random.shuffle(train_data)
        batches = minibatch(train_data, size=batch_sizes)
        for batch in batches:
            texts, annotations = zip(*batch)
            nlp.update(texts, annotations, sgd=optimizer, drop=attrs['dropout'], losses=losses)
        with textcat.model.use_params(optimizer.averages):
            # evaluate on the dev data split off in load_data()
            scores = evaluate(nlp.tokenizer, textcat, dev_texts, dev_cats)
        print(
            "{0:.3f}\t{1:.3f}\t{2:.3f}\t{3:.3f}".format(  # print a simple table
                losses["textcat"],
                scores["textcat_p"],
                scores["textcat_r"],
                scores["textcat_f"],
            ) 
        )

In [13]:
# test the trained model
test_text = 'The Lion King was very entertaining. The movie was visually spectacular.'
doc = nlp(test_text)
print(test_text)
print(doc.cats)

### 1.3 Register model for deployment

In [14]:
registered_model = client.get_or_create_registered_model(
    "spaCy Film Review Classifier",
)

Verta deployment expects a particular interface for its models.  
We'll create a thin wrapper class around our `spaCy` pipeline.

In [15]:
from verta.registry import VertaModelBase

class TextClassifier(VertaModelBase):
    def __init__(self, artifacts):
        with open(artifacts["nlp"], "rb") as f:
            self.nlp = pickle.load(f)

    def predict(self, input_list):  # param must be a list/batch of inputs
        predictions = []
        for text in input_list:
            scores = self.nlp(text).cats
            if scores['POSITIVE'] > scores['NEGATIVE']:
                predictions.append("POSITIVE")
            else:
                predictions.append("NEGATIVE")
        
        return predictions

Verta deployment also needs a couple more details about the model.

What do its inputs and outputs look like?

In [16]:
from verta.utils import ModelAPI  # Verta-provided utility class

input_list = [
    "This movie was subpar at best.",
    "Plot didn't make sense.",
]

model_api = ModelAPI(
    input_list,  # example inputs
    ["NEGATIVE", "NEGATIVE"],  # example outputs
)

What PyPI-installable packages (with version numbers) are required to deserialize and run the model?

In [17]:
from verta.environment import Python

environment = Python(["numpy", "spacy", "thinc"])

And finally, we'll register this version of our model into the Verta platform.

In [18]:
from verta.environment import Python

model_version = registered_model.create_standard_model(
    model_cls=TextClassifier,
    model_api=model_api,
    artifacts={"nlp": nlp},
    environment=environment,
    name="v1",
    attrs=attrs,
)

As a final validation, we can retrieve our model artifacts and run predictions locally.

In [19]:
artifacts = model_version.fetch_artifacts(["nlp"])
TextClassifier(artifacts).predict(input_list)

## 2. Deploy model to endpoint

In [20]:
endpoint = client.get_or_create_endpoint("film-review-classifier")
endpoint.update(model_version, wait=True)
deployed_model = endpoint.get_deployed_model()

In [21]:
deployed_model.predict(input_list)

In [22]:
deployed_model.predict(["I would definitely watch this again!"])

---